In [1]:
mkdir ~/. kaggle

mkdir: cannot create directory ‘/root/.’: File exists


In [2]:
cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
! chmod600 ~/.kaggle/kaggle.json

/bin/bash: line 1: chmod600: command not found


In [4]:
! kaggle datasets download -d aumthaker/mars-terrain-classification

Dataset URL: https://www.kaggle.com/datasets/aumthaker/mars-terrain-classification
License(s): MIT
100% 38.0M/38.1M [00:03<00:00, 17.3MB/s]
100% 38.1M/38.1M [00:03<00:00, 11.1MB/s]


In [5]:
! unzip mars-terrain-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: Auburn_1/crater/3975.jpg  
  inflating: Auburn_1/crater/3976.jpg  
  inflating: Auburn_1/crater/3984.jpg  
  inflating: Auburn_1/crater/3991.jpg  
  inflating: Auburn_1/crater/3995.jpg  
  inflating: Auburn_1/crater/3998.jpg  
  inflating: Auburn_1/crater/4.jpg   
  inflating: Auburn_1/crater/400.jpg  
  inflating: Auburn_1/crater/4005.jpg  
  inflating: Auburn_1/crater/4011.jpg  
  inflating: Auburn_1/crater/4018.jpg  
  inflating: Auburn_1/crater/4030.jpg  
  inflating: Auburn_1/crater/4031.jpg  
  inflating: Auburn_1/crater/4032.jpg  
  inflating: Auburn_1/crater/4040.jpg  
  inflating: Auburn_1/crater/4041.jpg  
  inflating: Auburn_1/crater/4045.jpg  
  inflating: Auburn_1/crater/405.jpg  
  inflating: Auburn_1/crater/4051.jpg  
  inflating: Auburn_1/crater/4073.jpg  
  inflating: Auburn_1/crater/4081.jpg  
  inflating: Auburn_1/crater/4090.jpg  
  inflating: Auburn_1/crater/4098.jpg  
  inflating: Auburn_1/crater/4102

In [6]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [7]:
imgen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [8]:
train = imgen.flow_from_directory(
    directory='/content/Auburn_1',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)
valid = imgen.flow_from_directory(
    directory='/content/Auburn_1',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

Found 4924 images belonging to 8 classes.
Found 1229 images belonging to 8 classes.


In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

In [11]:
model.fit(train, batch_size=32, epochs=20, validation_data=valid, callbacks=[early_stopping, reduce_lr])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


154/154 ━━━━━━━━━━━━━━━━━━━━ 24s 101ms/step - accuracy: 0.5507 - loss: 1.4533 - val_accuracy: 0.5956 - val_loss: 1.3013 - learning_rate: 0.0010
Epoch 2/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.5956 - loss: 1.2265 - val_accuracy: 0.6225 - val_loss: 1.0647 - learning_rate: 0.0010
Epoch 3/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - accuracy: 0.6494 - loss: 1.0832 - val_accuracy: 0.6444 - val_loss: 1.0690 - learning_rate: 0.0010
Epoch 4/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 13s 79ms/step - accuracy: 0.6995 - loss: 0.9144 - val_accuracy: 0.6965 - val_loss: 0.8972 - learning_rate: 0.0010
Epoch 5/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.7360 - loss: 0.7770 - val_accuracy: 0.7266 - val_loss: 0.8298 - learning_rate: 0.0010
Epoch 6/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.7726 - loss: 0.6512 - val_accuracy: 0.7331 - val_loss: 0.8533 - learning_rate: 0.0010
Epoch 7/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - accuracy: 0.8103 - loss: 0.5443

In [12]:
model.evaluate(valid)

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7140 - loss: 0.8433


[0.82975834608078, 0.7266070246696472]